# MCP (Model Context Protocol)

MCP is the "USB-C" for AI. Because it give every tool and data source one universal plug.

## Key Concepts:

* MCP: An open-source standard that lets AI models connect through one common plug

* Host: The main AI application (your laptop) that wants to use to the outside data

* Client: Component inside the host that its goin to open a connection to the outside world in order to connect to something

* Server: Offers actions to the client in a standarized way.

* Primitives: 5 different things that we can do, Prompts, Resource, Tools, Root, Sampling

## Diving deeper into MCP

Host (laptop that runs the AI experience that could be a chatbot in a browser), The host does not directly talk to the database or API, deligates that job to the clients -> Client is like a USB-C port, it is going to open a secure one conversation with the MCP server, this is called JSON-RPC over WebSocket, wich is fancy words for **simple real time messaging format** (you can have multiple clients but most often you are just going to have one).-> The clients are going to connect to servers through the MCP connection, the MCP is like the USB-C cable, and you can connect to Stripe, to a database, to some kind of documentation.

Servers Primitives:

* this are for instance text, or templates that the server can inject into the model context to guide the behaviour

* Tools: they are functions or things that the model can do

* Resources: read only documents, schemas filas, things that the model can reference.

* Roots: secure file system access that are set up by the client, so the model can only open the file that you allow.

* Sampling: it's a way or the server to ask the model for help.

## The N by N problem

It is hard to connect different models (even one) to stripe, email, CRM, database. They would all connect differently, they have different tools, and so on. And this is what we are solving, we are simplifying the ability for our AI to do stuff, to make stuff happen.

## Summary

MCP serves as the universal USB-C connector for AI, enabling any model to securely access and utilize various tools and data sources through a single, open standard protocol








Game Plan for MCP with OpenAI

What we are trying to do with MCP is that we allow the AI to connect to some tools some database to improve its own context.

We are allowing the AI, so our LLMs, to have this universal ports that allows to connect to the data, websites, platforms etc.

The AI will able to access to many stuff.

## MCP Servers

https://mcpservers.org/

https://mcpservers.org/servers/modelcontextprotocol/fetch

https://github.com/modelcontextprotocol/servers/tree/main/src/fetch

En mcpservers.org (o directorios similares como el oficial de Glama o Pulse), tú buscas "PostgreSQL" y encuentras un servidor listo para usar, mantenido por la comunidad o por los creadores oficiales.

**Un ejemplo de flujo de trabajo**

Imagina que quieres que tu IA tenga acceso a tus repositorios de GitHub.

Sin el directorio: Tendrías que leer la API de GitHub, autenticarte con OAuth, crear las funciones en Python y envolverlas en MCP.

Con mcpservers.org:

* Entras a la web.

* Buscas "GitHub".

* Copias la configuración del servidor oficial.

* Le das tu token de acceso.

* Listo: Tu IA ya puede crear Pull Requests o leer Issues.

## OpenAI MCP

https://platform.openai.com/docs/guides/tools-connectors-mcp

Esa guía es el manual de instrucciones para que los desarrolladores usen la API de OpenAI (específicamente la Responses API y el Agents SDK) para conectarse a herramientas externas usando el protocolo MCP.

# Setup

In [ ]:
# Set OpenAI key in the enviroment
from google.colab import userdata
import os

api_key = userdata.get('genai_course')
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
from openai import OpenAI
from IPython.display import MarkDown

client = OpenAI()

Web Browsing with Fetch

# MCP Fetch

In [ ]:
# Call the OpenAI API with the response endpoint
# https://mcpservers.org/remote-mcp-servers
response1 = client.response.create(
    model ='gpt-4.1',
    input = 'Find information on google maps and social media like reddit about the restaurant Rui dos Letioes in Coimbra, Portugal',
    tools = [
        {
          'type': 'mcp', # herramienta a utilizar es un servidor CMP
          'server_label': 'fetch',
          'server_url': 'https://remote.mcpservers.org/fetch/mcp',
          'require_approval': 'never' # buscará la información inmediatamente
        }
    ]
)

In [ ]:
# Display the output
Markdown(response1.output)

In [ ]:
# Explore the output
response1.output[0] # general details of what we are doing

In [ ]:
response1.output[1] # what the mcp did, the information

In [ ]:
response1.output[2] # number 2 is what was the actual output

In [ ]:
Markdown(response1.output[2].content[0].text)

In [ ]:
# What tools were used?
response1.tools

MCP Approval

# Requiere the approval

En la parte anterior, el modelo actuó solo. En esta sección, veremos cómo el código obliga al modelo a pedir permiso antes de hacer nada.

In [ ]:
# Call the OpenAI API with the response endpoint
# https://mcpservers.org/remote-mcp-servers
response2 = client.response.create(
    model ='gpt-4.1',
    input = 'Find information on google maps and social media like reddit about the restaurant Rui dos Letioes in Coimbra, Portugal',
    tools = [
        {
          'type': 'mcp',
          'server_label': 'fetch',
          'server_url': 'https://remote.mcpservers.org/fetch/mcp',
          'require_approval': 'always' # "No importa cuán seguro estés, detente antes de llamar a esta herramienta y pregúntame"
        }
    ]
)

In [ ]:
# Explore the putput with the approval
# the response2 now is different from response1
response2

In [ ]:
response2.output[0]

In [ ]:
# Approval request
# this is a two step aproach, where you dont have as much trust initially
response2.output[1]

Including Approval in API Call

El modelo está pausado esperando tu orden. El bloque response3 muestra cómo le contestamos programáticamente (esto simula cuando un usuario hace clic en "Aprobar" en una interfaz gráfica).

In [ ]:
# Include approval in the following conversation
response3 = client.response.create(
    model = 'gpt-4.1',
    tools = [
        {
            'type': 'mcp',
            'server_label': 'fetch',
            'server_url': 'https://remote.mcpservers.org/fetch/mcp',
        }
    ],
    input = [
        {
            'type': 'mcp_approval_response',
            'approve': True,
            'approval_request_id': response2.output[1].id
        }
    ],
    previous_response_id=response2.id
)

In [ ]:
# display the output of response3
Markdown(response3.output_text)

In [ ]:
# explore the output
response3

In [ ]:
response3.output # is a list

Resumen de esta sección: Pasamos de un modo "Autónomo" a un modo "Supervisado". Esto es vital si el servidor MCP tuviera capacidades peligrosas (como borrar_base_de_datos o enviar_dinero).

En una aplicación real (como una web o una app móvil), ese bloque de código que vimos (response3) es lo que se ejecuta después de que tú haces clic en un botón de "Aprobar" o "Confirmar".

Visualízalo así:

1. El Modelo se detiene: Al ver `require_approval`: 'always', el modelo genera el objeto `approval_request` y se queda esperando.

2. La Interfaz (UI) reacciona: Tu aplicación detecta esa pausa y te muestra una ventana emergente o un botón que dice:

> ⚠️ La IA quiere buscar en Google Maps. ¿Permitir? [ SÍ ] [ NO ]

3. Tú das la orden: Al hacer clic en [ SÍ ], tu aplicación envía internamente ese código `{'approve': True}`.

4. El Modelo continúa: Recibe tu permiso, ejecuta la herramienta, obtiene los datos del restaurante y termina su trabajo.

Es el equivalente moderno a cuando tu navegador te pregunta: "¿Quieres permitir que Google Maps acceda a tu ubicación?".

## ¿Por qué esto es crítico en ingeniería de IA?
* Seguridad: Evita que la IA borre datos, envíe emails o gaste dinero en APIs sin tu consentimiento explícito.

* Confianza: El usuario siente que tiene el control final sobre el agente.

Si tienes claro este concepto de "semáforo" (Rojo = Esperar aprobación, Verde = Ejecutar), estamos listos para la última parte del código, que es la más sofisticada: permisos selectivos.

Specifying Approvals: Deepwiki - Part 1


# Approvals for specific tools

In [ ]:
# Defining queries 3
query1 = 'Explain all the tools that this mcp can do'
query2 = 'Create complete documentation of the React Repo'
query3 = 'What is the React repo'

In [ ]:
# Call the api with deepwiki mcp
# goes through github repositories and explore
# https://docs.devin.ai/work-with-devin/deepwiki-mcp
response4 = client.response.create(
    model = 'gpt-4.1',
    input = query1,
    tools = [
        {
            'type': 'mcp',
            'server_label': 'deepwiki',
            'server_url': 'https://mcp.deepwiki.com/mcp',
            'require_approval': 'never'
        }
    ]
)

In [ ]:
Markdown(response4.output_text)

En response4, le estás dando permiso global al servidor deepwiki. Si ese servidor tiene 10 funciones diferentes (leer, escribir, borrar...), la IA puede usar cualquiera sin preguntar.

In [ ]:
response5 = client.response.create(
    model = 'gpt-4.1',
    input = query3,
    tools = [
        {
            'type': 'mcp',
            'server_label': 'deepwiki',
            'server_url': 'https://mcp.deepwiki.com/mcp',
            'require_approval': {
                'never': {'tool_names': ['ask_question']}
            }
        }
    ]
)

Estás definiendo una excepción de seguridad. La lógica que aplica el sistema es:

1. Regla General: Por defecto, asume que SÍ requiere aprobación para todo.

2. Excepción (never): "Sin embargo, para la herramienta específica llamada ask_question, NUNCA pidas aprobación".

¿Por qué hacer esto? (El caso de uso real)
Imagina que tienes un Servidor MCP conectado a tu Sistema de Archivos.

* Tiene una herramienta `read_file` (leer archivo).

* Tiene una herramienta `delete_file` (borrar archivo).

No quieres aprobar cada vez que la IA solo quiere leer algo (sería tedioso). Pero definitivamente quieres que te pregunte antes de borrar algo.

Con esta configuración:

* Si la IA decide usar `read_file` -> Se ejecuta automático.

* Si la IA decide usar `delete_file` -> Se pausa y aparece el botón de aprobación.

In [ ]:
Markdown(response5.output_text)

In [ ]:
# Explore the response 5 output
response5.output[0] # input
response5.output[1] # output

Resumen General del Código hasta ahora:

Este script te ha llevado por los tres niveles de madurez en la integración de MCP:

1. Nivel 1 (Confianza Total): `require_approval`: 'never'. Ideal para herramientas de solo lectura (búsquedas, clima).

2. Nivel 2 (Control Total): `require_approval`: 'always'. Ideal para herramientas críticas (pagos, envíos de emails).

3. Nivel 3 (Híbrido/Granular): Configuración por diccionario. Ideal para equilibrar una buena experiencia de usuario (UX) con la seguridad operativa.

Authentication with Stripe MCP - Part 1

# Authentication with Stripe

https://dashboard.stripe.com/login

In [ ]:
# get the stripe key

from google.colab import userdata
stripe_key = userdata.get('STRIPE_KEY')

In [ ]:
# Call the API with Stripe
response6 = client.responses.create(
    model = 'gpt-4.1',
    input = 'What are the tools in this mcp', # Create a customer called andrei with email andrei@gmail.com
    tools = [
        {
            'type': 'mcp',
            'server_label': 'stripe',
            'server_url': 'https://mcp.stripe.com',
            'headers': { # Autentificación para poder entrar
                'Authorization': f'Bearer {stripe_key}'
            }
        }
    ]
)

In [ ]:
# You can do a lot of things with stripe
Markdown(response6.output_text)

Authentication with Stripe MCP - Part 2

In [ ]:
# Explore the output
response6.output[0]

In [ ]:
response6.output[1]

In [ ]:
# Create a flow for approval
def stripe_request(query: str):
  request =  client.responses.create(
      model = 'gpt-4.1',
      input = query,
      tools = [
          {
              'type': 'mcp',
              'server_label': 'stripe',
              'server_url': 'https://mcp.stripe.com',
              'headers': {
                  'Authorization': f'Bearer {stripe_key}'
              }
          }
      ]
  )
  print(request.output[1])

  # Ask the user if the request is accepted or not
  approval = input('Do you want to approve the request? (y/n)')
  if approval == 'y':
    # Aprove the request
    # llamar de nuevo a client.response.create
    approval_request = client.response.create(
        model = 'gpt-4.1',
        tools = [
            {
                'type': 'mcp',
                'server_label': 'stripe',
                'server_url': 'https://mcp.stripe.com',
                'headers': {
                    'Authorization': f'Bearer {stripe_key}'
                }
            }
        ],
        previous_response_id = request.id,
        input = [
            {
                'type': 'mcp_approval_response',
                'approve': True,
                'approval_request_id': request.output[1].id
            }
        ]
    )
  return Markdown(approval_request)

In [ ]:
# Apply yh function
approval_request = stripe_request('Create a customer called Sebastian with email sebas@gmail.com')

In [ ]:
approval_request

Cuando se ejecuta la última línea: `approval_request = stripe_request('Create a customer called Sebastian...')`

Ocurre esto:

1. El LLM analiza tu texto y decide usar la herramienta `stripe.create_customer`.

2. El script se detiene y te muestra: "Voy a crear un cliente con email sebas@gmail.com".

3. El script te pregunta: `Do you want to approve the request? (y/n)`.

4. Tú escribes y y das Enter.

5. El script envía la señal de aprobación.

6. El servidor MCP de Stripe recibe la orden y realmente crea el usuario en tu base de datos de Stripe.